In [91]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.datasets import load_iris

# 1) Cargar datos en X,y - DATASET IRIS
iris = load_iris()
# Crear DataFrame con nombres apropiados
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris['species'] = iris.target
df_iris['species_name'] = [iris.target_names[i] for i in iris.target]  # Para mostrar los nombres

display(df_iris.head().style.set_caption("<h3>Primeras filas del dataset Iris<h3>"))

target = "species"  
X = df_iris.drop(columns=[target, 'species_name'])  # Eliminamos species y species_name
y = df_iris[target]  # Target: especies (0, 1, 2)

display(pd.DataFrame(X.head()).style.set_caption("<h3>Características (X)<h3>"))

display(pd.DataFrame({
    target: iris.target_names,
    'Código': range(len(iris.target_names))
}).style.set_caption("<h3>Target (y) <h3>"))
        
display(pd.DataFrame({
    'Variables': X.columns.tolist()
}).style.set_caption("<h3>Nombre de las características <h3>").hide(axis='index'))

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_name
0,5.100000,3.500000,1.400000,0.200000,0,setosa
1,4.900000,3.000000,1.400000,0.200000,0,setosa
2,4.700000,3.200000,1.300000,0.200000,0,setosa
3,4.600000,3.100000,1.500000,0.200000,0,setosa
4,5.000000,3.600000,1.400000,0.200000,0,setosa


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.100000,3.500000,1.400000,0.200000
1,4.900000,3.000000,1.400000,0.200000
2,4.700000,3.200000,1.300000,0.200000
3,4.600000,3.100000,1.500000,0.200000
4,5.000000,3.600000,1.400000,0.200000


,species,Código
0,setosa,0
1,versicolor,1
2,virginica,2


Variables
sepal length (cm)
sepal width (cm)
petal length (cm)
petal width (cm)


In [92]:
# 2) Codificar y normalizar
# Para Iris no necesita codificación porque ya son numéricas
# Las características ya están en escala similar (cm)

# 3) Separar
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4) Seleccion de modelo(s) 
bosque = RandomForestClassifier(random_state=42)

# 5) Entrenar
bosque.fit(X_train, y_train)

# 6) Probar y evaluar
y_pred = bosque.predict(X_test)

# Resultados en TEST
# Matriz de confusión
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=iris.target_names,
    index=iris.target_names
).style.set_caption("<h4>Matriz de confusión</h4>"))

# Reporte de clasificación
display(pd.DataFrame(
    classification_report(y_test, y_pred, target_names=iris.target_names, output_dict=True)
).transpose().style.set_caption("<h4> Reporte de clasificación </h4>"))

,setosa,versicolor,virginica
setosa,15,0,0
versicolor,0,14,1
virginica,0,4,11


,precision,recall,f1-score,support
setosa,1.000000,1.000000,1.000000,15.000000
versicolor,0.777778,0.933333,0.848485,15.000000
virginica,0.916667,0.733333,0.814815,15.000000
accuracy,0.888889,0.888889,0.888889,0.888889
macro avg,0.898148,0.888889,0.887767,45.000000
weighted avg,0.898148,0.888889,0.887767,45.000000


In [93]:
# 7) Validación Cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": "f1_macro",
    "roc_auc": "roc_auc_ovr",
    "average_precision": "average_precision",
}

scores = cross_validate(bosque, X_train, y_train, cv=cv, scoring=scoring, return_train_score=True, n_jobs=-1)

In [94]:
# Todas las métricas en una sola tabla
display(pd.DataFrame({
    'Métrica': [
        'Exactitud Validación',
        'Exactitud Entrenamiento', 
        'F1-Score Macro',
        'ROC AUC Medio',
        'Average Precision Medio'
    ],
    'Valor': [
        f"{scores['test_accuracy'].mean():.3f} ± {scores['test_accuracy'].std():.3f}",
        f"{scores['train_accuracy'].mean():.3f} ± {scores['train_accuracy'].std():.3f}",
        f"{scores['test_f1_macro'].mean():.3f} ± {scores['test_f1_macro'].std():.3f}",
        f"{scores['test_roc_auc'].mean():.3f}",
        f"{scores['test_average_precision'].mean():.3f}"
    ]
}).style.set_caption("<h3>VALIDACIÓN CRUZADA (5-FOLD)</h3>").hide(axis='index'))

Métrica,Valor
Exactitud Validación,0.952 ± 0.043
Exactitud Entrenamiento,1.000 ± 0.000
F1-Score Macro,0.952 ± 0.043
ROC AUC Medio,0.998
Average Precision Medio,0.995


In [95]:
# Todas las métricas en una sola tabla
display(pd.DataFrame({
    'Métrica': [
        'Métricas Principales',
        'Exactitud Validación',
        'Exactitud Entrenamiento', 
        'F1-Score Macro',
        'ROC AUC Medio',
        'Average Precision Medio'
    ],
    'Valor': [
        f"ACC {scores['test_accuracy'].mean():.3f} ± {scores['test_accuracy'].std():.3f} | F1 {scores['test_f1_macro'].mean():.3f} ± {scores['test_f1_macro'].std():.3f}",
        f"{scores['test_accuracy'].mean():.3f} ± {scores['test_accuracy'].std():.3f}",
        f"{scores['train_accuracy'].mean():.3f} ± {scores['train_accuracy'].std():.3f}",
        f"{scores['test_f1_macro'].mean():.3f} ± {scores['test_f1_macro'].std():.3f}",
        f"{scores['test_roc_auc'].mean():.3f}",
        f"{scores['test_average_precision'].mean():.3f}"
    ]
}).style.set_caption("<h3>VALIDACIÓN CRUZADA (5-FOLD)</h3>").hide(axis='index'))

Métrica,Valor
Métricas Principales,ACC 0.952 ± 0.043 | F1 0.952 ± 0.043
Exactitud Validación,0.952 ± 0.043
Exactitud Entrenamiento,1.000 ± 0.000
F1-Score Macro,0.952 ± 0.043
ROC AUC Medio,0.998
Average Precision Medio,0.995


In [96]:
# 6) Probar (con el conjunto de test)
y_pred = bosque.predict(X_test)

# 7) Evaluar en el conjunto de entrenamiento
# Reporte y Matriz de Confusión
display(pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=iris.target_names,
    index=iris.target_names
).style.set_caption("<h4>Matriz de Confusión</h4>"))

display(pd.DataFrame(
    classification_report(y_test, y_pred, target_names=iris.target_names, output_dict=True)
).transpose().style.set_caption("<h4>Reporte de Clasificación</h4>"))

display(pd.DataFrame({
    'Métrica': ['Score del Modelo'],
    'Valor': [f"{bosque.score(X_test, y_test):.3f}"]
}).style.set_caption("<h4>Evaluación Final</h4>").hide(axis='index'))

,setosa,versicolor,virginica
setosa,15,0,0
versicolor,0,14,1
virginica,0,4,11


,precision,recall,f1-score,support
setosa,1.000000,1.000000,1.000000,15.000000
versicolor,0.777778,0.933333,0.848485,15.000000
virginica,0.916667,0.733333,0.814815,15.000000
accuracy,0.888889,0.888889,0.888889,0.888889
macro avg,0.898148,0.888889,0.887767,45.000000
weighted avg,0.898148,0.888889,0.887767,45.000000


Métrica,Valor
Score del Modelo,0.889


# Resumen

In [97]:
display(pd.DataFrame({
    'Métrica': ['Accuracy Global', 'Validación Cruzada (ACC)', 'F1-Score Macro', 'ROC AUC'],
    'Valor': ['88.9%', '95.2% ± 4.3%', '95.2% ± 4.3%', '99.8%'],
    'Evaluación': [
        "Alta efectividad en clasificación",
        "Excelente consistencia entre folds", 
        "Desempeño balanceado entre clases",
        "Capacidad discriminativa excepcional"
    ]
}).style.set_caption("<h1>📈 RESUMEN EJECUTIVO DE PERFORMANCE<h1>").hide(axis='index'))

display(pd.DataFrame({
    'Especie': ['setosa', 'versicolor', 'virginica'],
    'Precisión': ['100%', '77.8%', '91.7%'],
    'Recall': ['100%', '93.3%', '73.3%'], 
    'F1-Score': ['100%', '84.8%', '81.5%'],
    'Diagnóstico': [
        "Clasificación perfecta",
        "Buena detección con algunas imprecisiones",
        "Alta precisión con algunos no detectados"
    ]
}).style.set_caption("<h3>🌿 ANÁLISIS DETALLADO POR ESPECIE<h3>").hide(axis='index'))

display(pd.DataFrame({
    'Especie': ['setosa', 'versicolor', 'virginica'],
    'Comportamiento': [
        "15/15 correctos - Clasificación perfecta",
        "14/15 correctos - 1 confundido con virginica", 
        "11/15 correctos - 4 confundidos con versicolor"
    ],
    'Patrón de Error': [
        "Ningún error de clasificación",
        "Leve confusión con especie virginica",
        "Confusión moderada con especie versicolor"
    ]
}).style.set_caption("<h3>🎯 INTERPRETACIÓN MATRIZ DE CONFUSIÓN<h3>").hide(axis='index'))

display(pd.DataFrame({
    'Aspecto': ['Generalización', 'Sobreajuste', 'Consistencia', 'Capacidad Discriminativa'],
    'Resultado': [
        "95.2% de efectividad en datos no vistos",
        "Brecha mínima entrenamiento vs validación", 
        "Alta consistencia (±4.3% entre folds)",
        "Excepcional (ROC AUC 99.8%)"
    ],
    'Evaluación': [
        "Excelente capacidad de generalización",
        "Modelo bien ajustado sin sobreajuste",
        "Performance estable y confiable",
        "Separa casi perfectamente las clases"
    ]
}).style.set_caption("<h3>🔄 ANÁLISIS DE VALIDACIÓN CRUZADA<h3>").hide(axis='index'))

display(pd.DataFrame({
    'Hallazgo': [
        "Clasificación perfecta para setosa",
        "Confusión entre versicolor y virginica", 
        "Performance de validación superior a test",
        "Capacidad discriminativa excepcional"
    ],
    'Implicación': [
        "El modelo identifica sin errores la especie setosa",
        "Las especies versicolor y virginica presentan similitudes",
        "El modelo generaliza mejor de lo que sugiere el test único",
        "El modelo casi perfecto separando las clases"
    ]
}).style.set_caption("<h3>🎯 CONCLUSIONES Y HALLAZGOS PRINCIPALES<h3>").hide(axis='index'))

Métrica,Valor,Evaluación
Accuracy Global,88.9%,Alta efectividad en clasificación
Validación Cruzada (ACC),95.2% ± 4.3%,Excelente consistencia entre folds
F1-Score Macro,95.2% ± 4.3%,Desempeño balanceado entre clases
ROC AUC,99.8%,Capacidad discriminativa excepcional


Especie,Precisión,Recall,F1-Score,Diagnóstico
setosa,100%,100%,100%,Clasificación perfecta
versicolor,77.8%,93.3%,84.8%,Buena detección con algunas imprecisiones
virginica,91.7%,73.3%,81.5%,Alta precisión con algunos no detectados


Especie,Comportamiento,Patrón de Error
setosa,15/15 correctos - Clasificación perfecta,Ningún error de clasificación
versicolor,14/15 correctos - 1 confundido con virginica,Leve confusión con especie virginica
virginica,11/15 correctos - 4 confundidos con versicolor,Confusión moderada con especie versicolor


Aspecto,Resultado,Evaluación
Generalización,95.2% de efectividad en datos no vistos,Excelente capacidad de generalización
Sobreajuste,Brecha mínima entrenamiento vs validación,Modelo bien ajustado sin sobreajuste
Consistencia,Alta consistencia (±4.3% entre folds),Performance estable y confiable
Capacidad Discriminativa,Excepcional (ROC AUC 99.8%),Separa casi perfectamente las clases


Hallazgo,Implicación
Clasificación perfecta para setosa,El modelo identifica sin errores la especie setosa
Confusión entre versicolor y virginica,Las especies versicolor y virginica presentan similitudes
Performance de validación superior a test,El modelo generaliza mejor de lo que sugiere el test único
Capacidad discriminativa excepcional,El modelo casi perfecto separando las clases
